In [1]:
import re
import glob
import math
from math import inf
from math import log
from math import exp

#凝固度遍历需要cut 大于N的情况
#自由度也是

## 语料生成器

In [2]:
# 语料生成器，并且初步预处理语料
# 这个生成器例子的具体含义不重要，只需要知道它就是逐句地把文本yield出来就行了
def text_generator():
    '''每次选出一篇文章,每次返回一篇文章的处理后的字符串的一行'''
    #txts_path = glob.glob('corpus/short1/*.txt')#匹配所有文章
    #txts_path = glob.glob('corpus/THUCNews/THUCNews/eco/79*.txt')
    #txts_path = glob.glob('corpus/corporation/*.txt')
    # txts_path = glob.glob('corpus/corporation/nt_names_short.txt')
    txts_path = glob.glob('corpus/corporation/nt_names.txt')
    for txt_path in txts_path:#每次选出一篇文章，对这一篇文章处理后,返回一篇文章的字符串
        f=open(txt_path,'r',encoding='utf-8')
        s = f.readline()
        while s:
            s = s.replace(u'\u3000', ' ').strip() #把中文空格转为英文空格
            s = re.sub(' ','',s) #把空格转为空
            s = re.sub(u'[^\u4e00-\u9fa50-9a-zA-Z ]+', '\n', s)#将特殊字符转换为换行
            s=s.split('\n')
            for i in range(len(s)):
                yield s[i] #每次返回一行
            s = f.readline()



## 统计词频

In [3]:

class TrieNode:
    '''特征树节点'''
    def __init__(self) -> None:
        self.next={}
        self.count=0 #记录词出现的次数
        self.solid=inf #记录词的凝固度
        #self.entrophy_right=-1 #记录词的左熵
        #self.entrophy_left=-1 #记录词的右熵
        self.free_degree=-1 #记录词的自由度(左熵右熵计算而得) 可以不记录左熵和右熵 记录的原因是可以看左熵和右熵的值
    def print_solid(self,print_num=200):
        '''打印print_num个词的凝固度 以凝固度降序排序'''
        anslist=[]
        root=self
        def dfs(root,word):
            if len(word)>1:#只记录两字以上词的凝固度
                anslist.append([word,root.solid])
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        anslist.sort(key=lambda x:-x[1])#按照凝固度降序排序
        print('自由度')
        for i in range(print_num):
            print(anslist[i])
    
    def print_free_degree(self,print_num=200):
        '''打印print_num个词的自由度,左熵,右熵 以自由度降序排序'''
        anslist=[]
        root=self
        def dfs(root:TrieNode,word):
            if len(word)>1:
                if hasattr(self,'entrophy_left'):
                    anslist.append([word,root.free_degree,root.entrophy_left,root.entrophy_right])
                else:
                    anslist.append([word,root.free_degree])
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        anslist.sort(key=lambda x:-x[1])#按自由度降序排序
        print('自由度,左熵,右熵')
        for i in range(print_num):
            print(anslist[i])

    def print_solid_and_free_degree(self,print_num=200,descent='solid'):
        '''打印print_num个词的凝固度,自由度,左熵,右熵 以descent降序排序'''
        'descent可以取值为solid 或 free_degree'
        if descent=='solid':
            descent=1
        elif descent=='free_degree':
            descent=2
        else:
            descent=1
        anslist=[]
        root=self
        def dfs(root:TrieNode,word):
            if len(word)>1:
                if hasattr(self,'entrophy_left'):
                    anslist.append([word,root.solid,root.free_degree,root.entrophy_left,root.entrophy_right])
                else:
                    anslist.append([word,root.solid,root.free_degree])
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        anslist.sort(key=lambda x:-x[descent])#按自由度降序排序
        print('凝固度,自由度,左熵,右熵')
        for i in range(print_num):
            print(anslist[i])
    

class NgramCount_t:
    '''统计词频,最多只统计长度为N,并且出现次数大于min_count次的词语'''
    def __init__(self,N=4,min_count=3) -> None:
        self.root=TrieNode()#根节点的count记录字数
        self.root.solid=-1 #根节点没有凝固度
        self.N=N #统计词的最大长度(在计算自由度时需考虑N+1个字)
        self.min_count=min_count #统计词的最少数量
        self.letterNum=0 #统计单个字的个数
    def do_count(self,txt):
        '''给定一个txt文本,统计词频'''
        root=self.root
        sentences=txt.split('\n')
        for sentence in sentences:
            for i in range(len(sentence)):
                node=root
                for j in range(i,min(i+self.N+1,len(sentence))):#记录N+1个字
                    ch=sentence[j]
                    if ch not in node.next:
                        node.next[ch]=TrieNode()
                    node=node.next[ch]
                    node.count+=1
                self.letterNum+=1
        self.root.count=self.letterNum
    def cut_tree(self,min_count=None):
        '''只保留树中出现次数大于等于min_count次的节点'''
        if min_count==None:
            min_count=self.min_count
        root=self.root
        def dfs(root,pre,curch):
            if 0<root.count<min_count:
                del root
                del pre.next[curch]
                return
            for ch in list(root.next.keys()):
                node=root.next[ch]
                dfs(node,root,ch)
        dfs(root,None,None)
    
    def print_count(self,min_count=None,print_all=False):
        '''打印出现次数大于等于min_count的词典,如果print_all,则打印所有词语'''
        '''如果min_count为1即打印所有词'''
        if min_count==None:
            min_count=self.min_count
        if print_all:
            min_count=1
        root=self.root
        ansdict={}
        def dfs(root,word):
            if root.count>=min_count:
                ansdict[word]=root.count
            #已经次数小于min_count的词 后续节点一定小于min_count
            elif root.count!=0:
                return
            if len(word)==self.N:
                return
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        print(ansdict)

    

## 计算凝固度

In [4]:
#凝固度计算
class solid_calc_t:
    '''给词典的特征树根节点,凝固度阈值,限定词长N'''
    '''min_proba 字数对应凝固度边界 凝固度= p(x,y)/(p(x)*p(y)) = total*n(x,y)/(n(x)*n(y)) '''
    '''凝固度越大 x和y越相关 凝固度越小 x和y越独立'''
    '''N为最大只统计长度'''
    def __init__(self,root:TrieNode,min_proba={2:5, 3:25, 4:125},N=4) -> None:
        self.root=root
        self.min_proba = min_proba
        self.min_proba={2:1.2, 3:20, 4:30}
        self.N=N

        #处理min_proba中值不足的情况
        if len(self.min_proba)+1<self.N:
            if 2 not in self.min_proba:
                self.min_proba[2]=5
            for i in range(3,self.N+1):
                if i not in self.min_proba:
                    self.min_proba[i]=self.min_proba[i-1]*5
        
    def calc_solid(self):
        '''计算凝固度,返回凝固度符合词的凝固度和列表'''
        anslist=[]
        root=self.root

        def find_word_count(word):
            '''计算一个词出现的次数'''
            node=root
            for ch in word:
                if ch not in node.next:
                    print(f'word:{word} not in Dictionary')
                    return 0
                node=node.next[ch]
            return node.count

        def calc_back_fromCurNode(root,left_word):
            '''以当前root节点为分割点,计算所有后面节点的凝固度,left_word为当前节点代表的词语'''
            def dfs_calc(root:TrieNode,right_word,left_count,left_word):
                right_count=find_word_count(right_word) #计算分割点右侧词出现的次数
                word_count=root.count #当前整个词出现的次数
                solid=self.root.count*word_count/(left_count*right_count) #计算凝固度 total*n(x,y) / (n(x)*n(y))
                root.solid=min(root.solid,solid)

                if len(right_word)+len(left_word)==self.N:
                    return
                for ch,node in root.next.items():
                    dfs_calc(node,right_word+ch,left_count,left_word)

            for ch,node in root.next.items():
                dfs_calc(node,ch,root.count,left_word)


        def dfs(root,word):
            '''遍历所有节点,计算后面所有节点的凝固度'''
            if len(word)>0:
                calc_back_fromCurNode(root,word)
                if len(word)>=2 and root.solid>=self.min_proba[len(word)]:#词长度大于等于2 且词的凝固度符合要求
                    anslist.append([word,root.solid])
            if len(word)==self.N:#计算凝固度只计算最多N个字的情况
                return
            for ch,node in root.next.items():
                dfs(node,word+ch)

        dfs(root,'')
        anslist.sort(key=lambda x:-x[1])
        return anslist

    def print_solid(self,print_num=200):
        '''打印print_num个节点的凝固度'''
        anslist=[]
        def dfs(root,word):
            if len(word)>1:#只记录两字以上词的凝固度
                anslist.append([word,root.solid])
            if len(word)==self.N:#计算凝固度只计算最多N个字的情况
                return
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(self.root,'')
        anslist.sort(key=lambda x:-x[1])#按照凝固度降序排序
        print('凝固度')
        print(anslist[:print_num])



In [5]:
#词频统计
nc_forward=NgramCount_t() #用于计算凝固度 和右熵
nc_backward=NgramCount_t() #用于计算左熵
txt_generator=text_generator()
print('finish0')
cnt=0 #记录读取行数
cutcnt=200000 #多少行切割一次树
epoch=10 #运行次数 如果全部运行 调成epoch=-1即可
min_count=4 #删去出现次数少于min_count的词
#txt_generator=['四是四十一是十十四是十四四十一是四十一']
for txt in txt_generator:
    cnt+=1
    if cnt%cutcnt==0:
        print(txt)
        nc_forward.cut_tree(min_count)#删去少于min_count的词
        nc_backward.cut_tree(min_count)#删去少于min_count的词
        print(f'write {cnt//cutcnt}*{cutcnt} rows')
        if cnt==cutcnt*epoch:
            break
    
    nc_forward.do_count(txt)#将文章字加入计数
    nc_backward.do_count(txt[::-1])#将文章颠倒字加入计数

nc_forward.cut_tree(min_count)#删去少于min_count的词
nc_backward.cut_tree(min_count)#删去少于min_count的词
print('finish1')

#nc_forward.print_count(min_count=3,print_all=False)#打印出现次数大于等于min_count的词
print(nc_forward.letterNum)#打印总共的字数




finish0
咸安区旺旺副食店
write 1*200000 rows
合肥唐氏建材经营部
write 2*200000 rows
上海凡华贸易有限公司
write 3*200000 rows
北京博骏贸易有限公司
write 4*200000 rows
山东宏忠轴承有限公司
write 5*200000 rows
柯坪县卫生健康委员会
write 6*200000 rows
湖州慈爱康复服务中心
write 7*200000 rows
长治市城区三好食品厂
write 8*200000 rows
北安市第三中学学生食堂
write 9*200000 rows
广州市穗福建材有限公司
write 10*200000 rows
finish1
19170665


In [6]:
sc=solid_calc_t(nc_forward.root) #创建计算凝固度类
sc.calc_solid() #计算凝固度

[['徘徊', 3834133.0],
 ['岣嵝', 3834133.0],
 ['鄌郚', 2738666.4285714286],
 ['癫痫', 1917066.5],
 ['蛤蟆', 958533.25],
 ['旮旯', 947999.9175824176],
 ['哪吒', 931907.3263888889],
 ['疼痛', 760740.6746031746],
 ['摔跤', 684666.6071428572],
 ['邂逅', 580929.2424242424],
 ['琵琶', 547733.2857142857],
 ['ABB', 495793.0603448276],
 ['蜘蛛', 471659.2182539682],
 ['犯罪', 456444.40476190473],
 ['蜻蜓', 425664.4366776316],
 ['汫洲镇汫', 399388.8541666667],
 ['鸳鸯', 342333.3035714286],
 ['浛洸', 319511.0833333333],
 ['贷款', 303093.5177865613],
 ['秸秆', 301178.48019125685],
 ['烹饪', 299541.640625],
 ['诱惑', 291861.0857371795],
 ['鲇鱼须', 285277.7529761905],
 ['篱笆', 276333.9099099099],
 ['饸饹', 252245.59210526315],
 ['浐灞', 242666.64556962025],
 ['癌症', 189184.19407894736],
 ['壕堑', 182577.7619047619],
 ['斑竹垱', 182577.7619047619],
 ['蘑菇', 181616.82631578948],
 ['烔炀', 176902.68819815444],
 ['AB', 176281.97701149425],
 ['溶解乙炔', 175877.6605504587],
 ['氨酯', 154246.72988505746],
 ['睡眠', 152551.71087533157],
 ['888', 145232.3106060606],
 ['疯狂', 1

In [7]:
sc.print_solid(print_num=100)

凝固度
[['十字', 3682.5], ['戚墅堰', 2946.0], ['施工队', 2455.0], ['委员', 2455.0], ['监督', 2455.0], ['研究', 2455.0], ['集团', 2104.285714285714], ['铸造', 2104.285714285714], ['旅游', 1841.25], ['轴承', 1841.25], ['股权', 1841.25], ['柏坊', 1753.5714285714287], ['线缆', 1683.4285714285713], ['传媒', 1636.6666666666667], ['精密', 1636.6666666666667], ['初级', 1636.6666666666667], ['执法', 1636.6666666666667], ['墅堰', 1636.6666666666667], ['戚墅', 1636.6666666666667], ['百货', 1578.2142857142858], ['辉埠', 1402.857142857143], ['烟洲', 1339.090909090909], ['印刷', 1227.5], ['幼儿', 1133.076923076923], ['办公室', 1133.076923076923], ['街道办', 1133.076923076923], ['溧阳', 1133.076923076923], ['红十', 1052.142857142857], ['红十字', 1052.142857142857], ['荫田', 920.625], ['西岭', 906.4615384615385], ['环境', 866.4705882352941], ['保洁', 866.4705882352941], ['道办事处', 866.4705882352941], ['体育', 849.8076923076923], ['街道', 841.7142857142857], ['街道办事', 818.3333333333334], ['道办事', 818.3333333333334], ['运输', 818.3333333333334], ['生态', 775.2631578947369], ['卫生', 775.26

## 计算自由度

In [15]:
#自由度(左右熵)计算
class free_degree_calc_t:
    '''根据正向词典和反向词典计算左右熵'''
    def __init__(self,root_forward,root_backward,N=4) -> None:
        self.root_forward=root_forward #正向词典根节点
        self.root_backward=root_backward #反向词典根节点
        self.N=N
    
    def calc_word_free_degree(self,word):
        '''计算一个词的左熵和右熵,返回自由度,左熵,右熵'''

        entrophy_right=self.calc_word_entrophy(self.root_forward,word) #计算右熵
        entrophy_left=self.calc_word_entrophy(self.root_backward,word[::-1]) #计算左熵 需要在反向字典中查找反向词
        free_degree=self.calc_free_degree_by_SmoothNlp(entrophy_left,entrophy_right) #计算自由度

        return free_degree,entrophy_left,entrophy_right
        
    def calc_words_free_degree(self):
        '''遍历词典,计算所有词左熵和右熵'''
        root_forward=self.root_forward
        root_backward=self.root_backward

        
            
        def dfs(root,word):
            
            if len(word)>=2:
                #print('左熵,右熵,自由度')
                entrophy_right=self.calc_node_entrophy(root) #计算右熵
                entrophy_left=self.calc_word_entrophy(self.root_backward,word[::-1]) #计算左熵 需要在反向字典中查找反向词
                #print(word,entrophy_left,entrophy_right,end=' ')
                free_degree=self.calc_free_degree_by_SmoothNlp(entrophy_left,entrophy_right) #计算自由度
                #print(free_degree)
                root.entrophy_right=entrophy_right #左熵和右熵可以不记录
                root.entrophy_left=entrophy_left #
                root.free_degree=free_degree
                
            if len(word)==self.N:#计算自由度只计算最多N个字的情况
                return   

            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root_forward,'')
    def calc_node_entrophy(self,root:TrieNode):
        '''给定一个节点,计算该节点在树中的右熵,返回熵值'''
        cnt=0 #节点右邻字 字的总数
        entrophy=0 #熵值
        for node in root.next.values():
            cnt+=node.count
        
        for node in root.next.values():
            p=node.count/cnt
            entrophy+=p*self.calc_self_information(p)
        
        
        return entrophy


    def calc_word_entrophy(self,root:TrieNode,word):
        '''给定一个词典树和一个词,计算该词在树中的右熵'''
        '''用反向特征树词典计算的其实是该词在原正向树中的左熵'''
        node=root
        for ch in word:
            #print(ch)
            if ch not in node.next:
                print(f'word:{word} not in found root')
                return -1
            node=node.next[ch]
        entrophy=self.calc_node_entrophy(node)
        return entrophy

    
    def calc_self_information(self,p):
        '''给定一个概率,计算自信息的值'''
        return -math.log(p)
    
    def calc_free_degree_by_SmoothNlp(self,entrophy_left,entrophy_right):
        '''使用SmoothNlp计算公式在已知左熵和右熵的情况下计算自由度'''
        EL=entrophy_left
        ER=entrophy_right
        if EL==ER:
            return EL
        if EL==0:
            return ER
        #由于文本的特殊性 想要结尾 而右熵可能未出现
        if ER==0:
            return EL
        return log((EL*exp(ER)+ER*exp(EL))/abs(EL-ER))

    def calc_free_degree_by_HelloNLP(self,entrophy_left,entrophy_right):
        '''使用HelloNLP计算公式在已知左熵和右熵的情况下计算自由度'''
        #这个计算方法貌似有问题
        EL=entrophy_left
        ER=entrophy_right
        if EL==0 and ER==0:
            return 1e-50
        if EL==ER:
            return EL
        if EL==0:
            EL=1e-100
        if ER==0:
            ER=1e-100
        ret=ER*log(EL/abs(EL-ER))+EL*log(ER/abs(EL-ER))
        return ret

    def print_free_degree(self,print_num=200):
        '''打印词的凝固度,自由度,左熵,右熵 以自由度降序排序'''
        anslist=[]
        root=self.root_forward
        def dfs(root:TrieNode,word):
            if len(word)>1:
                anslist.append([word,root.solid,root.free_degree,root.entrophy_left,root.entrophy_right])
            if len(word)==self.N:#计算凝固度只计算最多N个字的情况
                return
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')

        anslist.sort(key=lambda x:-x[2])#按自由度降序排序
        print('凝固度','自由度,左熵,右熵')
        for i in range(print_num):
            print(anslist[i])

In [8]:
fdc=free_degree_calc_t(nc_forward.root,nc_backward.root)
fdc.calc_words_free_degree()


In [9]:
def check(root,word):
    node=root
    for ch in word:
        node=node.next[ch]
    return node

def print_word_solid(root:TrieNode,word):
    node=root
    for ch in word:
        if ch not in node.next:
            print(f'word:{word} not in root')
        node=node.next[ch]
    print(node.solid)
word='有限公司'
nn=check(nc_forward.root,word)
print(f'word:{word}')
print('右熵',fdc.calc_word_entrophy(fdc.root_forward,word))
print('左熵',fdc.calc_word_entrophy(fdc.root_backward,word[::-1]))
print('凝固度,自由度,左熵,右熵,出现次数')
print_word_solid(fdc.root_forward,word)
print(fdc.calc_word_free_degree(word))
print(nn.count)


word:有限公司
右熵 0.8086966907017401
左熵 3.300514795711712
凝固度,自由度,左熵,右熵,出现次数
19.696078780162896
(2.4661702886956207, 3.300514795711712, 0.8086966907017401)
945180


In [10]:
fdc.print_free_degree(print_num=200)

凝固度 自由度,左熵,右熵
['县华', 2.355989567133082, 13.645013448144454, 4.038315395939571, 4.038858948496193]
['县孔', 10.987094909704199, 10.997813829677762, 1.9632566703885, 1.9627887247725926]
['市新', 3.7502009076848126, 10.969122129963134, 4.159265716712319, 4.1501433291906995]
['县谷', 1.1025518430413357, 10.829970862137495, 1.768640928776445, 1.768230493616871]
['电信', 4.9340523348462755, 10.566478199404179, 2.085127950177321, 2.085992991816984]
['县向阳', 12.838334157907372, 10.20275450411224, 1.58833063107639, 1.5877544350633455]
['州鑫', 2.753940294356683, 9.970234999887742, 2.825751372855073, 2.83022510063957]
['红十字会', 218.77042182544324, 9.841201384307599, 1.2673341348707559, 1.2678133428548217]
['县大', 3.200460318877554, 9.8288572656468, 4.388325918528835, 4.427317722642378]
['装璜', 465.51810204433724, 9.720203870372318, 1.690168815841001, 1.6912702050012398]
['县隆', 1.470016782169271, 9.64688994655448, 2.2983455475103343, 2.2953937053278834]
['利达', 8.244665761876984, 9.488459006436502, 2.1828069201

In [15]:
nc_forward.root.print_solid_and_free_degree(descent='solid')

凝固度,自由度,左熵,右熵
['十字', 3682.5, 0.0]
['州市戚墅堰', 2946.0, -1]
['戚墅堰', 2946.0, 0.0]
['常宁市柏坊', 2455.0, -1]
['常山县辉埠', 2455.0, -1]
['施工队', 2455.0, 0]
['委员', 2455.0, 0]
['监督', 2455.0, 0]
['研究', 2455.0, 0]
['常宁市西岭', 2356.8, -1]
['集团', 2104.285714285714, 0]
['铸造', 2104.285714285714, 0]
['旅游', 1841.25, 0]
['轴承', 1841.25, 0]
['股权', 1841.25, 0]
['柏坊', 1753.5714285714287, 0.0]
['线缆', 1683.4285714285713, 0]
['常州市戚墅', 1636.6666666666667, -1]
['传媒', 1636.6666666666667, 0]
['精密', 1636.6666666666667, 0]
['初级', 1636.6666666666667, 0]
['执法', 1636.6666666666667, 0]
['墅堰', 1636.6666666666667, 0.0]
['戚墅', 1636.6666666666667, 0.0]
['百货', 1578.2142857142858, 0]
['辉埠', 1402.857142857143, 0.0]
['常宁市烟洲', 1339.090909090909, -1]
['烟洲', 1339.090909090909, 0.0]
['印刷', 1227.5, 0]
['常州市溧阳', 1133.076923076923, -1]
['幼儿', 1133.076923076923, 0]
['办公室', 1133.076923076923, 0]
['街道办', 1133.076923076923, 0]
['溧阳', 1133.076923076923, 0.0]
['常宁市洋泉', 1052.142857142857, -1]
['红十', 1052.142857142857, 0.0]
['红十字', 1052.142857142857, 0.

In [11]:
class word_solid_free_degree_filter_t():
    '''根据凝固度和自由度筛选词语'''
    def __init__(self,root,min_proba={2:5, 3:25, 4:125},N=4) -> None:
        self.root=root
        self.min_proba=min_proba
        self.N=N

        #处理min_proba中值不足的情况
        if len(self.min_proba)+1<self.N:
            if 2 not in self.min_proba:
                self.min_proba[2]=5
            for i in range(3,self.N+1):
                if i not in self.min_proba:
                    self.min_proba[i]=self.min_proba[i-1]*5
    
    def filter_solid(self):
        '''对凝固度不满足的词语进行去除'''
        def check_child_isDelete(root:TrieNode,word_length,rootch,word):
            '''检查所有孩子节点是否可以删除,如果可以,删除所有孩子节点,保留父节点root。word_length为到root节点的词的长度'''
            '''rootch为当前root所代表的字'''
            '''word为当前root所代表的词'''
            if word_length>self.N:#已经到叶子节点,叶子节点不能直接删除
                return False
            

            if word_length>=2 and root.solid>=self.min_proba[word_length]: #父节点凝固度满足条件 则无需删除所有子节点
                return False
            else: #父节点凝固度不满足条件
                isDelete=True
                if word_length==self.N:#已经是最长词的情况 直接删除所有子节点即可
                    pass
                else:#查看所有子节点是否满足凝固度要求
                    for ch,node in root.next.items():
                        if node.solid>=self.min_proba[word_length+1]:#有子节点不能删 那么父节点不能删 那么其他子节点也不能删除
                            return False
                #所有节点都不满足凝固度要求 删除所有子节点
                for ch,node in list(root.next.items()):
                    print(word+ch,word_length+1)
                    del node
                    del root.next[ch]
                
                if word_length==1:#词长为1的词已经没有子节点 可以删去
                    print(word,word_length)
                    del root
                    del self.root.next[rootch]
                return True

        root=self.root
        def dfs(root:TrieNode,word_length,rootch,word):
            for ch,node in list(root.next.items()):
                dfs(node,word_length+1,ch,word+ch)
            #遍历到最底层后开始反向删除
            if word_length!=0:#根节点不进行判断删除
                check_child_isDelete(root,word_length,rootch,word)
        dfs(root,0,'','')
    
    def select_valid_words(self,min_proba={2:5, 3:25, 4:125},free_degree_proba=0,count_limit=10000,print_num=inf):
        '''根据给定的凝固度阈值和自由度阈值对符合的词进行筛选'''
        anslist=[]
        root=self.root
        def dfs(root:TrieNode,word):
            if len(word)>self.N:
                return
            if 2<=len(word)<=self.N:
                if root.solid>=min_proba[len(word)] and root.free_degree>free_degree_proba and root.count>=count_limit:
                    anslist.append([word,root.solid,root.free_degree,root.count])
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        print('词,凝固度,自由度,出现次数')
        for i in range(min(len(anslist),print_num)):
            print(anslist[i])
        return anslist
    
    

        
    

In [12]:
wsfdf=word_solid_free_degree_filter_t(nc_forward.root)
#wsfdf.filter_solid()


In [32]:
def check(root,word):
    node=root
    for ch in word:
        node=node.next[ch]
    return node

def print_word_solid(root:TrieNode,word):
    node=root
    for ch in word:
        if ch not in node.next:
            print(f'word:{word} not in root')
        node=node.next[ch]
    print(node.solid)
word='分公司'
nn=check(nc_forward.root,word)
print(f'word:{word}')
print('右熵',fdc.calc_word_entrophy(fdc.root_forward,word))
print('左熵',fdc.calc_word_entrophy(fdc.root_backward,word[::-1]))
print('凝固度,自由度,左熵,右熵,出现次数')
print_word_solid(fdc.root_forward,word)
print(fdc.calc_word_free_degree(word))
print(nn.count)


KeyError: '公'

In [16]:
wsfdf.select_valid_words(free_degree_proba=1,min_proba={2:5,3:17, 4:125},count_limit=1000)

词,凝固度,自由度,出现次数
['东方', 32.029789739414596, 4.887440902624298, 2674]
['东莞', 170.92035773829485, 1.6890934978154772, 11427]
['东莞市', 36.228180443773915, 2.9492561122824585, 9956]
['东省', 24.011178827785074, 3.0412151944051513, 4063]
['北京', 129.6010567681192, 5.196733753707757, 74915]
['北省', 9.659366056798177, 2.650281455823022, 1705]
['大学', 7.397564503121431, 5.253421791697373, 3622]
['大丰', 9.917080232440497, 2.458109348937892, 1160]
['大理', 8.866491696461882, 1.8032062001798477, 1449]
['大连', 190.92067414563473, 2.2207219035815084, 11808]
['大酒', 33.666556447532564, 1.1586594966443753, 2060]
['大酒店', 75.4436274163628, 3.645253613284971, 1961]
['大药', 39.94595663387051, 1.1214414193112316, 2092]
['大同', 66.46118989141823, 2.9425914158123274, 2364]
['大同市', 28.81112965453178, 1.9931409091792214, 1638]
['大庆', 15.65812288797586, 1.7463051063013233, 2047]
['大队', 73.89995649956435, 2.7168435138360194, 3704]
['学院', 11.11872781839992, 4.280126304611586, 4784]
['学校', 117.0819384315155, 4.313540591860834, 

[['东方', 32.029789739414596, 4.887440902624298, 2674],
 ['东莞', 170.92035773829485, 1.6890934978154772, 11427],
 ['东莞市', 36.228180443773915, 2.9492561122824585, 9956],
 ['东省', 24.011178827785074, 3.0412151944051513, 4063],
 ['北京', 129.6010567681192, 5.196733753707757, 74915],
 ['北省', 9.659366056798177, 2.650281455823022, 1705],
 ['大学', 7.397564503121431, 5.253421791697373, 3622],
 ['大丰', 9.917080232440497, 2.458109348937892, 1160],
 ['大理', 8.866491696461882, 1.8032062001798477, 1449],
 ['大连', 190.92067414563473, 2.2207219035815084, 11808],
 ['大酒', 33.666556447532564, 1.1586594966443753, 2060],
 ['大酒店', 75.4436274163628, 3.645253613284971, 1961],
 ['大药', 39.94595663387051, 1.1214414193112316, 2092],
 ['大同', 66.46118989141823, 2.9425914158123274, 2364],
 ['大同市', 28.81112965453178, 1.9931409091792214, 1638],
 ['大庆', 15.65812288797586, 1.7463051063013233, 2047],
 ['大队', 73.89995649956435, 2.7168435138360194, 3704],
 ['学院', 11.11872781839992, 4.280126304611586, 4784],
 ['学校', 117.081938431515

In [128]:
fdc.print_free_degree(print_num=200)

凝固度 自由度,左熵,右熵
['六必居', 1776.2422310094641, 37.82900699995622, 1.0466304865832219, 1.0466304865832217]
['星月', 35.82676323514934, 13.01693986914496, 1.6548975813683686, 1.654936070867217]
['县彭', 6.893601844079685, 12.815765737932297, 2.5782977354585355, 2.5781131355131333]
['谦和', 24.6303673093485, 12.001184526820197, 1.0852280400722045, 1.0851886129676505]
['区首', 1.4439381356620586, 11.863351687183027, 0.6930128020601277, 0.6929932791588755]
['县均', 7.127818771711665, 11.764487800336708, 1.4378692508404116, 1.4377750789545047]
['惠民', 18.584238684404358, 11.68109175199794, 2.587855383538344, 2.5884374906033845]
['高新区东', 1.6579280347015615, 11.644558936616434, 1.0974088280586503, 1.0973511772469036]
['崇明大', 4.310913828639335, 11.644558936616434, 1.0974088280586503, 1.0973511772469036]
['理工大学', 162.98004904930738, 11.508978897569165, 2.141159699311533, 2.140793961081198]
['县全', 1.960809964898034, 11.277677177652896, 2.3026360960651022, 2.3020536401234146]
['区中原', 1.7744783037503171, 11.263916

In [138]:
def print_solid_and_free_degree(root,print_num=200,descent='solid'):
        '''打印print_num个词的凝固度,自由度,左熵,右熵 以descent降序排序'''
        'descent可以取值为solid 或 free_degree'
        
        if descent=='solid':
            descent=1
        elif descent=='free_degree':
            descent=2
        else:
            descent=1
        anslist=[]
        
        def dfs(root:TrieNode,word):
            if len(word)>1:
                if hasattr(root,'entrophy_left'):
                    anslist.append([word,root.solid,root.free_degree,root.entrophy_left,root.entrophy_right])
                else:
                    anslist.append([word,root.solid,root.free_degree])
            for ch,node in root.next.items():
                dfs(node,word+ch)
        dfs(root,'')
        anslist.sort(key=lambda x:-x[descent])#按自由度降序排序
        print('凝固度,自由度,左熵,右熵')
        for i in range(print_num):
            print(anslist[i])

In [139]:
print_solid_and_free_degree(nc_forward.root,print_num=1000,descent='solid')

凝固度,自由度,左熵,右熵
['胳膊', 37395671.75, 0.0, 0.0, 0.0]
['源黑果花楸', 24930447.833333332, -1]
['圐圙', 24930447.833333332, 0, 0, 0]
['里水镇贤僚', 18697835.875, -1]
['九州通讬嵄', 16620298.555555556, -1]
['江佳友领呔', 16620298.555555556, -1]
['讬嵄', 16620298.555555556, 0.0, 0.0, 0.0]
['东乡区珀玕', 13598426.090909092, -1]
['道治污减霾', 13598426.090909092, -1]
['定市佁琳騺', 10547497.16025641, -1]
['佁琳騺', 10547497.16025641, 5.328025050104764e-05, 0.0, 0.6931471805599453]
['禾甸镇大淜', 10460327.762237763, -1]
['栾城区窦妪', 9972179.133333333, -1]
['自治县桠杈', 9348917.9375, -1]
['衡阳县岣嵝', 8798981.588235294, -1]
['岣嵝', 8798981.588235294, 0.0, 0.0, 0.0]
['落潭镇寮寀', 8499016.306818182, -1]
['沙河市褡裢', 8310149.277777778, -1]
['褡裢', 8310149.277777778, 0.0, 0.0, 0.0]
['台州小背篓', 7872773.0, -1]
['pa', 7479134.35, 0, 0, 0]
['昌乐县鄌郚', 6799213.045454546, -1]
['厦门市筼筜', 6799213.045454546, -1]
['草宾川白肋', 6799213.045454546, -1]
['筼筜', 6799213.045454546, 0.0, 0.0, 0.0]
['席梦思床褥', 6503595.0869565215, -1]
['鄌郚', 6043744.929292929, 0.0, 0.0, 0.0]
['co', 5949311.41477272

In [133]:

print(123)

123
